In [1]:
import cv2
import mediapipe as mp

In [1]:
# Initialize MediaPipe Pose solution.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize MediaPipe drawing utility.
mp_drawing = mp.solutions.drawing_utils

# Capture video from the default camera.
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect the pose.
    results = pose.process(image)

    print(results.x, results.y, results.z)

    """
    results object contains the x y and z axes values of the point that will be drawn
                                visibility number which indicated whether that point is clear
    """


    # Draw the pose annotations on the image.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the resulting image.
    cv2.imshow('MediaPipe Pose', image)

    # Break the loop if 'q' is pressed.
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

pose.close()
cap.release()


NameError: name 'mp' is not defined

### Pytorch LSTM (haven't checked if it works)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class FallDetectionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(FallDetectionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


### Preparing the input (haven't checked if it works)

In [ ]:
import numpy as np

In [ ]:
# Parameters
sequence_length = 30  # Length of the sequence to be fed into the LSTM
num_joints = 33       # Number of joints detected by MediaPipe Pose
num_features = 4      # x, y, z, and visibility for each joint

# Initialize a list to store pose data
pose_data = []

# Process each frame (example in a loop, replace with actual frame processing)
for frame in frames:
    results = pose.process(frame)
    if results.pose_landmarks:
        frame_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        pose_data.append(frame_data)

        # Check if collected enough data
        if len(pose_data) == sequence_length:
            # Convert to numpy array and reshape for LSTM input
            lstm_input = np.array(pose_data).reshape(1, sequence_length, -1)  # Shape: (1, sequence_length, num_features*num_joints)
            pose_data.pop(0)  # Remove the oldest frame data to maintain the sequence length

            # Convert to PyTorch tensor
            lstm_input_tensor = torch.tensor(lstm_input, dtype=torch.float32)

            # Now you can feed this tensor into your LSTM model
            # model_output = model(lstm_input_tensor)
            # Perform your fall detection logic based on the model_output
